In [22]:
#https://www.kaggle.com/nicapotato/universal-sentence-encoder-semantic-similarity

In [2]:
import os
import re
import sys
import numpy as np
import pandas as pd
rootfold = %pwd
rootfold

'/home/j0hndoe/Documents/git/reddit-disinformation/3_embed'

In [77]:
sys.path.append(rootfold+"/../2_findflags/all_subreddits")
import functions_pos_match as fpm

In [270]:
from importlib import reload  
reload(fpm)

<module 'functions_pos_match' from '/home/j0hndoe/Documents/git/reddit-disinformation/3_embed/../2_findflags/all_subreddits/functions_pos_match.py'>

### Test specific phrases

In [274]:
phrases = ['Removed due to fox news being an unreliable source.',
           'Fox News is an unreliable source.',
           'STOP spreading misinformation and panic.',
           'Ahhh yes, more propaganda.',
           'Unreliable source - of course left this out:',
           'Clickbait bullshit instead of investigative journalism.',
           'Sloppy, bullshit headline, CTV. ',
           'author spreads fake news',
           'Do not fall for propaganda',
           'BS title']

In [275]:
[(t.lower_, t.lemma_, t.pos_, t.dep_)  for d in phrases for t in fpm.nlp(d)]

[('removed', 'remove', 'VERB', 'acomp'),
 ('due', 'due', 'ADP', 'prep'),
 ('to', 'to', 'ADP', 'pcomp'),
 ('fox', 'fox', 'PROPN', 'compound'),
 ('news', 'news', 'NOUN', 'pobj'),
 ('being', 'be', 'AUX', 'ROOT'),
 ('an', 'an', 'DET', 'det'),
 ('unreliable', 'unreliable', 'ADJ', 'amod'),
 ('source', 'source', 'NOUN', 'attr'),
 ('.', '.', 'PUNCT', 'punct'),
 ('fox', 'Fox', 'PROPN', 'compound'),
 ('news', 'News', 'PROPN', 'nsubj'),
 ('is', 'be', 'AUX', 'ROOT'),
 ('an', 'an', 'DET', 'det'),
 ('unreliable', 'unreliable', 'ADJ', 'amod'),
 ('source', 'source', 'NOUN', 'attr'),
 ('.', '.', 'PUNCT', 'punct'),
 ('stop', 'STOP', 'PROPN', 'nsubj'),
 ('spreading', 'spread', 'VERB', 'ROOT'),
 ('misinformation', 'misinformation', 'NOUN', 'dobj'),
 ('and', 'and', 'CCONJ', 'cc'),
 ('panic', 'panic', 'NOUN', 'conj'),
 ('.', '.', 'PUNCT', 'punct'),
 ('ahhh', 'ahhh', 'VERB', 'ROOT'),
 ('yes', 'yes', 'INTJ', 'intj'),
 (',', ',', 'PUNCT', 'punct'),
 ('more', 'more', 'ADJ', 'amod'),
 ('propaganda', 'propaganda'

In [276]:
for sent_index,sent in enumerate(fpm.nlp.pipe(phrases, n_process=10, n_threads = 20)):
    currentmatch = [(sent_index, sent.text, m ) for m in sent[:]._.structurematched]
    print(currentmatch)

[(0, 'Removed due to fox news being an unreliable source.', 'stvao')]
[(1, 'Fox News is an unreliable source.', 'xvao')]
[(2, 'STOP spreading misinformation and panic.', 'stvo')]
[]
[]
[(5, 'Clickbait bullshit instead of investigative journalism.', 'o')]
[]
[(7, 'author spreads fake news', 'yvao')]
[(8, 'Do not fall for propaganda', 'stvo')]
[(9, 'BS title', 'ao')]


### Test manually classified phrases

In [277]:
MANUAL = pd.read_csv("../4_check/data/test_classified.csv")
MANUAL["class3"]=MANUAL["class"]
MANUAL.head()

,id,index,word,subm_title,domain,comm_body,class,disinformation,fakenews,bs,misleading,unreliable,propaganda,sample,flag,POSflag,class3
0,1200,119742,propaganda,"Spain bought 640,000 testing kits from China. ...",elpais.com,PROPAGANDA,f,0,0,0,0,0,4,pos,4,1,f
1,1199,122382,propaganda,WHO Says It Cannot Invite Taiwan to Annual Sum...,news.yahoo.com,Presenting one country's foreign policy as a N...,n,0,0,0,0,0,1,all,0,0,n
2,1198,72974,propaganda,Why Australia must not bow to China but seek w...,theage.com.au,I listened to the Chinese radio station based ...,n,0,0,0,0,0,1,all,0,0,n
3,1197,112251,propaganda,Chinese Scholar arrested by CCP after calling ...,scmp.com,"No way man, this sounds like Western Imperial ...",n,0,0,0,0,0,1,pos,1,1,n
4,1196,181332,propaganda,Trump pushes his party to normalize corruption,washingtonpost.com,Washington Post is a rabid leftist propaganda ...,f,0,0,0,0,0,1,all,1,1,f


In [263]:
disinfoFN = MANUAL.query('disinformation==1 & POSflag==0 & class3 == "f"')
for text in fpm.nlp.pipe(disinfoFN.comm_body, n_process=10, n_threads = 20, disable = ["ner"]):
    print(text)
    print('----------------------\n')
    for sent in text.sents:
        print(sent)
        currentmatch = [(sent.text, m) for m in sent[:]._.structurematched]
        print(currentmatch)
    print('\n\n')

The clip they're quoting from the interview says this "Are you sorry you sent Rudy Giuliani to Ukraine, your personal lawyer, are you sorry you did that?" Trump: "No, not at all" Trump already admitted to sending Giuliani to Ukraine to root out corruption. The part that has been contested was whether it was specifically to root out corruption on the Bidens. This isn't new info. The trial already brought out the fact that Trump sent Giuliani to Ukraine - this interview quoted didn't specify it was to "find damaging information about his political opponents". I hate Trump and I think he should have been removed from office for what he's done (and has done since), but this is a highly misleading title. It's not what was said. Let's all be better people than Fox news and stop spreading misinformation when there's already too much going around. There's enough awful stuff to report on about Trump, we don't need to embellish anything.
----------------------

The clip they're quoting from the 

In [278]:
newFLAG = []
for text in fpm.nlp.pipe(MANUAL.comm_body, n_process=10, n_threads = 20, disable = ["ner"]):
    hit=0
    for sent in text.sents:
        currentmatch = [(sent.text, m) for m in sent[:]._.structurematched]
        hit+=len(currentmatch)
    newFLAG+=[hit]

In [279]:
MANUAL['newFLAG'] = newFLAG
MANUAL.newFLAG.value_counts()

1    605
0    498
2     82
3     12
4      3
Name: newFLAG, dtype: int64

In [266]:
for i, m in MANUAL.iterrows():
    if m.newFLAG == 0 and m.class3 =="f":
        print(m.comm_body)
        print('\n')

The war propaganda is starting up


Saying that China didn't help Italy is factually incorrect, whatever you might believe about their true motives. Calling cooperation "warfare" is emotionally charged propaganda.


&gt;That moment may be closer and more real than we realise, one where China can play a central role. In the next two years we have an opportunity to mend a broken system, restore a damaged planet, rebuild the social contract in nations, and reinvent the global governance system. I truly enjoy chinese propaganda.


Lol, literal propaganda, this gets called out in all the UK papers comments sections or BBC comments, but Redditors love it, really proves the redditor = retard trope


Vitamin C has been shown in studies to reduce length and severity of pneumonia. Could care less about this propaganda. Vitamin C has saved my life a few times.


I’m all for bashing the ccp but should we really listen to Murdoch propaganda to do so. I mean fuck, this may as well be foxnews or brei

In [94]:
for i, m in MANUAL.iterrows():
    if m.newFLAG == 0 and m.POSflag>0:
        print(m.comm_body)
        print('\n')

1. DuPont isn't a PPE company. They make a huge variety of industrial products, only a tiny % of their product is PPE. Misleading representation of what DuPont is. That's like calling WalMart a PPE company because they sell masks and gloves. 2. DuPont's stock lost over 50% of it's value from the January(over $60/share)meeting referenced to March(down to $28/share). Not sure how buying a tanking stock is "convenient" as the title calls it. Of course no mention of this in the article. They want to trick stupid people into believing he's making money from insider trading. 3. The public knew about the coronavirus and there were confirmed cases in the US by the time of the meeting. Insinuating he got inside information is ignorant 4. He made 1.8 million in investments. During the time frame. Only 10% 180k was invested in DuPont. No mention of high ranking Democrat Feinstein selling 6 million in stocks right before the market crash 🤔. I guess that doesn't fit Vice's leftist propaganda


Oh t

In [267]:
for i, m in MANUAL.iterrows():
    if m.newFLAG > 0 and m.POSflag==0:
        print(m.comm_body)
        print('\n')

I listened to the Chinese radio station based in Sydney 2CR radio for Chinese community and the morning program was a lady talking about China and Australia and she was extremely bias. She said things like Australia is small little potato and will lose this trade and shouldn’t be a dog following the US and China is great bla bla bla. This station is a Chinese propaganda machine and please be wary of it


This push to the center is propaganda from the billionaire media owners. They want to not pay taxes and far left groups would tax them


Just for everyone’s info, CGTN is a propaganda machine news network straight from China.


Why are we posting articles from the CCP propaganda system? This.paper is a propaganda arm of the CCP


More fuel for their base I guess. Just a propaganda news outlet defending a propaganda YouTube video that was telling people to question experts and not wear masks during a global pandemic with a respiratory virus.


Removed due to fox news being an unreliable

In [280]:
MANUAL[['id','index','newFLAG']].to_csv("../4_check/data/check_newFLAG.csv", index = False)

### Older

In [3]:
import tensorflow as tf
import tensorflow_hub as hub

#module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
module_url = rootfold+"/model/universal-sentence-encoder_4"
ebmeddingmodel = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return ebmeddingmodel(input)
def tf_simil(s1,s2): return np.inner(s1,s2)
def tf_simil_matrix(embeddings, rd = 8): return [[round(tf_simil(s1,s2),rd) for s1 in embeddings] for s2 in embeddings]

module /home/j0hndoe/Documents/git/reddit-disinformation/3_embed/model/universal-sentence-encoder_4 loaded


In [4]:
sentfiles = sorted([st for st in os.listdir(rootfold+'/../output/') if re.search('MATCH_2020_',st)])

In [5]:
flagtypes = ['disinformation','fakenews','bs','misleading','unreliable','propaganda']

In [6]:
allsents=pd.DataFrame()
existing_docids = set()
mean_flag_embeddings = dict()
for file in sentfiles:
    SENT = pd.read_csv(rootfold+'/../output/'+file)
    allsents = allsents.append(SENT, ignore_index = True)
    existing_docids = existing_docids.union(set(SENT.comm_id))
for flgtype in flagtypes:
    s = allsents[allsents[flgtype]>0]
    mean_flag_embeddings[flgtype] = tf.reduce_mean(embed(s[s.flag>0].sent), axis = 0)

In [7]:
SENT.sent = SENT.sent.str.replace('’','\'')

In [8]:
allsents2 = allsents.copy()
allsents2.sent = allsents2.sent.astype('str').str.lower().str.replace('’',"'")

In [9]:
len(allsents2)

516825

### Re-evaluate POS in SENTS

In [87]:
del sys.modules["functions_pos_match"]
import functions_pos_match as fpm

In [88]:
allsents.head()

,comm_id,sent_id,sent,flag,disinformation,fakenews,bs,misleading,unreliable,propaganda
0,t1_fcty1dr,2,corbyn and co made mistakes in the campaign wh...,0,1,0,0,0,0,0
1,t1_fcv1kkn,0,"this is obviously good news, but i think the h...",0,0,0,0,1,0,0
2,t1_fcuqb2t,0,"can't rely on actual data, so the guardian has...",0,0,0,1,0,0,0
3,t1_fctstvk,0,more bullshit about democrats needing republic...,1,0,0,1,0,0,0
4,t1_fcts6fx,1,; vote center neoliberal bs,1,0,0,1,0,0,0


In [ ]:
allmatches=[]
for sent_index, sent in enumerate(fpm.nlp.pipe(allsents2.sent, n_process=10, n_threads = 20, disable = ["ner"])):
    currentmatch = [(sent_index, sent.text, m ) for m in sent[:]._.structurematched]
    allmatches += currentmatch
    if sent_index % 50000 == 1:
        print('%d extracted / %d index' % (len(allmatches), sent_index))

In [582]:
allmatches_df = pd.DataFrame(allmatches, columns = ['sent_index', 'sent', 'matchtype' ])
#allmatches_df['sent'] = [s.text for s in allmatches_df.sent]
allmatches_df.to_csv(rootfold+'/../output/comm_matches_06_02.csv')
allmatches_df.shape

(64752, 3)

New classification

In [90]:
# Only if not rerun
allmatches_df = pd.read_csv(rootfold+'/../output/comm_matches_06_02.csv')
allmatches_df

,Unnamed: 0,sent_index,sent,matchtype
0,0,9,the guise that it was to save jobs is pure bs.,svo
1,1,10,this is pure bs.,svo
2,2,13,/cv!recreate.action?pid=1410032001&amp;selecte...,o
3,3,24,that title is at least a little misleading whe...,sva
4,4,26,terrible clickbait headline.,ao
...,...,...,...,...
64747,64747,373499,this fascist bullshit?,o
64748,64748,373500,"so, on one level, the article title could be a...",sva
64749,64749,373505,the article is a little bit misleading.,sva
64750,64750,373513,https://www.reddit.com/r/canada/comments/fme6w...,stvao


In [91]:
freq_matches = allmatches_df.matchtype.value_counts()
freq_matches

svo       17605
sva       12918
o         10978
ao         8550
svao       4815
stvao      3271
vo         2044
stvo       1801
ivo        1213
vao         380
naao        336
yvo         257
ivsva       161
ivsvao      125
yvao         94
ivsvo        71
svsv         57
ivao         38
svsav        22
svna         14
svna2         2
Name: matchtype, dtype: int64

In [437]:
freq_matches = allmatches_df.matchtype.value_counts()
freq_matches

svo       15702
sva       12188
o         10702
ao         7068
svao       4338
stvao      3253
vo         1997
stvo       1775
ivo        1177
vao         369
naao        335
yvo         253
ivsva       161
ivsvao      124
yvao         95
ivsvo        68
svsv         52
ivao         39
svsav        22
svna         14
svna2         2
Name: matchtype, dtype: int64

In [434]:
allmatches_df2.matchtype.value_counts()

svo       15476
sva       11775
o         10702
ao         6799
svao       4031
stvao      3209
vo         1997
stvo       1348
vao         369
yvo         252
svsvo       226
svsvao      110
yvao         95
aost         20
svna         14
ost           5
svna2         2
Name: matchtype, dtype: int64

In [584]:
allmatches_df2 = allmatches_df.copy()
allmatches_df2.sent = allmatches_df2.sent.str.replace(r'http\S+\s?','', regex=True)
allmatches_df2.sent = [str(s)[0:70] for s in allmatches_df2.sent]
for matchtype in freq_matches.index:
    print(matchtype+' ('+ str(freq_matches[matchtype]) +' hits)'+'\n----------')
    print(allmatches_df2[allmatches_df2.matchtype == matchtype].sent.value_counts()[:50])
    print('\n\n')
    with open(rootfold+'/flag_types.txt','a') as file:
        file.write(matchtype+' ('+ str(freq_matches[matchtype]) +' hits)'+'\n----------\n')
        file.write(str(allmatches_df2[allmatches_df2.matchtype == matchtype].sent.value_counts()[:25]))
        file.write('\n\n')

svo (17605 hits)
----------
it may also be /r/noshitsherlock or clickbait that belongs in /r/saved    1411
this is bullshit.                                                          518
that's bullshit.                                                           234
it's bullshit.                                                             224
it's all bullshit.                                                          97
this is such bullshit.                                                      94
this is propaganda.                                                         82
this is bullshit                                                            68
it's propaganda.                                                            59
this article is bullshit.                                                   52
this is complete bullshit.                                                  50
this is misinformation.                                                     49
this is fucking bullshit

yvo (257 hits)
----------
why are you spreading misinformation?                                     14
you are spreading misinformation.                                         14
you're spreading misinformation.                                          12
you are spreading misinformation                                           4
why are you spreading disinformation?                                      3
how about you post this without the bullshit clickbait title               2
dude, you spreading misinformation /u                                      2
you are spreading misinformation on a topic about misinformation!          2
why are you spreading chinese government propaganda?                       2
are you seriously posting this propaganda bullshit from china daily??      2
you are spewing propaganda.                                                2
why are you spreading bullshit?                                            2
like you don't know this, you still post this fake

In [585]:
allsents['flag2'] = 0
allsents.loc[pd.unique(allmatches_df.sent_index),'flag2'] = 1
allsents2 = allsents.copy()
allsents2.sent = allsents2.sent.str.replace(r'http\S+\s?','', regex=True)
allsents2.sent = [str(s)[:150] for s in allsents2.sent]
pd.pivot_table(allsents2, index = ['flag'], columns = ['flag2'], values = 'sent', aggfunc='count')

flag2,0,1
flag,,
0,310913,19482
1,3079,40050


Posts that used to be flagged but are no longer flagged

In [586]:
allsents2[(allsents2.flag==1) & (allsents2.flag2==0)].sent.value_counts()[:50]

Your post has been removed because it has a sensationalized, editorialized, or biased headline and is therefore in violation of [Submission Rule #4](     76
Quit your bullshit.                                                                                                                                       21
’s bullshit.                                                                                                                                              21
Stop with this bullshit.                                                                                                                                  13
Stop with the bullshit.                                                                                                                                   12
Stop the bullshit.                                                                                                                                        10
Stop your fake news OP                                    

Posts that are still not flagged

In [587]:
allsents2[(allsents2.flag==0) & (allsents2.flag2==0)].sent.value_counts()[:50]

Unverified twitter/youtube accounts, facebook pages, or just general unverified personal accounts are not acceptable.                                     43844
Users often report submissions from this site and ask us to ban it for sensationalized articles.                                                           5337
If you do find evidence that this article or its title are false or misleading, contact the moderators who will review it                                  5337
*comments will be removed if they are disrespectful, off-topic, or spread misinformation                                                                   2300
Our community will not be used to spread misinformation during this important time.                                                                        1521
Is blatant misinformation or rumors that could cause a panic.                                                                                              1521
Hello everyone, we would just like to re

### Compute similarity

In [342]:
sent_simil = {flgtype:pd.DataFrame(columns = ['sent','flag','simil']) for flgtype in flagtypes}
mean_flag_embeddings = dict()

for file in sentfiles:
    SENT = pd.read_csv(rootfold+'/../output/'+file)
    SENT.sent = SENT.sent.str.lower().str.replace('’','\'')
    print(file)
    for flgtype in flagtypes:
        print('Started:' + flgtype)
        s = SENT[SENT[flgtype]>0]
        initial_flags_embedings = embed(s.sent)
        mean_flag_embedding = tf.reduce_mean(embed(s[s.flag>0].sent), axis = 0)
        simildf = pd.DataFrame([(sent,flag,tf_simil(e, mean_flag_embedding)) 
                                     for sent,flag,e in zip(s.sent, s.flag, initial_flags_embedings)
                                     if tf_simil(e, mean_flag_embedding)>0.4], 
                                    columns = ['sent','flag','simil']).\
                        sort_values('simil', ascending=False)
        sent_simil[flgtype] = sent_simil[flgtype].append(simildf)
        print('Done:' + flgtype)
    print('\n\n')

MATCH_2020_01.csv
Started:disinformation
Done:disinformation
Started:fakenews
Done:fakenews
Started:bs
Done:bs
Started:misleading
Done:misleading
Started:unreliable
Done:unreliable
Started:propaganda
Done:propaganda



MATCH_2020_02.csv
Started:disinformation
Done:disinformation
Started:fakenews
Done:fakenews
Started:bs
Done:bs
Started:misleading
Done:misleading
Started:unreliable
Done:unreliable
Started:propaganda
Done:propaganda



MATCH_2020_03.csv
Started:disinformation
Done:disinformation
Started:fakenews
Done:fakenews
Started:bs
Done:bs
Started:misleading
Done:misleading
Started:unreliable
Done:unreliable
Started:propaganda
Done:propaganda



MATCH_2020_04.csv
Started:disinformation
Done:disinformation
Started:fakenews
Done:fakenews
Started:bs
Done:bs
Started:misleading
Done:misleading
Started:unreliable
Done:unreliable
Started:propaganda
Done:propaganda



MATCH_2020_05.csv
Started:disinformation
Done:disinformation
Started:fakenews
Done:fakenews
Started:bs
Done:bs
Started:misle

### Most frequent similar that are not flags

In [343]:
del sys.modules["functions_pos_match"]
import functions_pos_match as fpm

#### Disinformation

In [344]:
s = sent_simil['disinformation']
print(s[s.flag==0].sent.value_counts()[:20])
print('\n\n\n')
print(s[s.flag==0][:50])

- spreading misinformation                                      142
don't spread misinformation.                                     41
please don't spread misinformation.                              23
don't spread misinformation                                      10
* spreading misinformation                                        9
stop spreading misinformation.                                    5
do not spread misinformation.                                     5
please do not spread misinformation.                              5
contain the spread of misinformation.                             4
quit spreading misinformation.                                    4
don't spread disinformation.                                      3
how is this misinformation?                                       3
don't spread misinformation please.                               3
i don't want to spread misinformation                             3
that's not disinformation.                      

In [345]:
for sent in fpm.nlp.pipe(s[s.flag==0].sent.value_counts()[:1000].index, n_process=10, n_threads = 20, disable = ["ner"]):
    matched_keywords = sent[:]._.structurematched
    if matched_keywords:
        print('%s | %s' % (sent, matched_keywords))

- spreading misinformation | ['o']
don't spread misinformation. | ['stvo']
please don't spread misinformation. | ['stvo']
don't spread misinformation | ['stvo']
stop spreading misinformation. | ['stvo']
do not spread misinformation. | ['stvo']
please do not spread misinformation. | ['stvo']
quit spreading misinformation. | ['stvo']
don't spread disinformation. | ['stvo']
don't spread misinformation please. | ['stvo']
i don't want to spread misinformation | ['stvo']
please don't spread this misinformation. | ['stvo']
please refrain from spreading misinformation. | ['stvo']
dont spread misinformation and fear. | ['stvo']
dont spread misinformation. | ['stvo']
*. do not spread misinformation. | ['stvo']
don't spread disinformation | ['stvo']
https://www.snopes.com/fact-check/matt-gaetz-mug-shot/ don't spread misinformation. | ['stvo']
bs btw stop spreading misinformation | ['stvo']
i think that's misinformation | ['svsvo']
i don't mean to spread more misinformation. | ['stvo']
oh boy, so 

In [346]:
for sent in fpm.nlp.pipe(s[s.flag==0].sent.value_counts()[:50].index, n_process=10, n_threads = 20, disable = ["ner"]):
    matched_keywords = sent[:]._.structurematched
    print('%s | %s' % (sent, matched_keywords))

- spreading misinformation | ['o']
don't spread misinformation. | ['stvo']
please don't spread misinformation. | ['stvo']
don't spread misinformation | ['stvo']
* spreading misinformation | []
stop spreading misinformation. | ['stvo']
do not spread misinformation. | ['stvo']
please do not spread misinformation. | ['stvo']
contain the spread of misinformation. | []
quit spreading misinformation. | ['stvo']
don't spread disinformation. | ['stvo']
how is this misinformation? | []
don't spread misinformation please. | ['stvo']
i don't want to spread misinformation | ['stvo']
that's not disinformation. | []
please don't spread this misinformation. | ['stvo']
a lot of misinformation going around. | []
please refrain from spreading misinformation. | ['stvo']
there's a lot of misinformation going around in this thread. | []
his misinformation is getting people killed. | []
misinformation is dangerous. | []
there is plenty of misinformation about withholding. | []
this is just spreading misinfo

#### Fake News

In [347]:
s = sent_simil['fakenews']
print(s[s.flag==0].sent.value_counts()[:20])
print('\n\n\n')
print(s[s.flag==0][:50])

this is fake.                                                                                                                                                                                                      61
it's fake.                                                                                                                                                                                                         59
fake news.                                                                                                                                                                                                         48
fake                                                                                                                                                                                                               42
congratulations for being tricked by fake news.                                                                                                 

In [348]:
for sent in fpm.nlp.pipe(s[s.flag==0].sent.value_counts()[:1000].index.str.lower(), n_process=10, n_threads = 20, disable = ["ner"]):
    matched_keywords = sent[:]._.structurematched
    if matched_keywords:
        print('%s | %s' % (sent, matched_keywords))

this is fake. | ['sva']
it's fake. | ['sva']
fake news. | ['ao']
fake | ['o']
it's all fake. | ['sva']
fake. | ['o']
this is fake | ['sva']
it's fake | ['sva']
not fake. | ['o']
fake news! | ['ao']
it's fake news. | ['svao']
so fake. | ['o']
fake news | ['ao']
its fake | ['o']
clearly fake. | ['o']
its fake. | ['o']
*fake | ['o']
the fake news new narrative is disgraceful &amp; false!” | ['o']
this is fake! | ['sva']
it's fake news | ['svao']
don't spread fake news. | ['stvao']
it's fake! | ['sva']
fake! | ['o']
it is fake. | ['sva']
it's a fake. | ['sva']
that's fake. | ['sva']
deep fake. | ['o']
fake news, this is just a document saying what to do if we run out of icus, this is * | ['ao']
fake? | ['o']
completely fake. | ['o']
totally fake. | ['o']
probably fake | ['o']
because it's fake. | ['sva']
it can be fake. | ['sva']
don't spread fake news | ['stvao']
it's all fake | ['sva']
this is fake news | ['svao']
fake and gay | ['o']
this is so fake | ['sva']
this is clearly fake. | ['s

In [349]:
for sent in fpm.nlp.pipe(s[s.flag==0].sent.value_counts()[:50].index.str.lower(), n_process=10, n_threads = 20, disable = ["ner"]):
    matched_keywords = sent[:]._.structurematched
    print('%s | %s' % (sent, matched_keywords))

this is fake. | ['sva']
it's fake. | ['sva']
fake news. | ['ao']
fake | ['o']
congratulations for being tricked by fake news. | []
it's all fake. | ['sva']
fake news is the deliberate attempt to publish hoaxes and/or disinformation for the purpose of profit or influence (learn more). | []
cnn is fake news. | []
you drive away anyone remotely persuadable by posting links from these bullshit, biased fake "news" sites, and reinforce your bubble wherein absolutely anyone who's politically aware thinks exactly like you. | []
fake trial, fake president. | []
you are fake news! | []
explainlife dot com is fake news. | []
fake. | ['o']
this is fake | ['sva']
next up, fake election. | []
it's fake | ['sva']
not fake. | ['o']
this is fucking fake. | []
cnn is fake news | []
it's not fake. | []
fake news! | ['ao']
you're fake news. | []
is this fake news? | []
it's fake news. | ['svao']
how is this fake news? | []
this story is sourced to the discredited mike adams from natural news, a well known

#### BS

In [303]:
s = sent_simil['bs']
print(s[s.flag==0].sent.value_counts()[:20])
print('\n\n\n')
print(s[s.flag==0][:50])

I call bullshit.             356
Bullshit.                    238
What a load of bullshit.      69
It’s bullshit.                68
I call bullshit               53
That’s bullshit.              49
It’s all bullshit.            38
This is fucking bullshit.     32
What a bullshit article.      30
I'm calling bullshit.         29
What a bunch of bullshit.     26
Fuck this bullshit.           24
I call bullshit!              18
Yeah, bullshit.               18
Which is bullshit.            17
Bullshit!                     17
I’m calling bullshit.         14
Fucking bullshit.             14
Calling bullshit.             13
I call bullshit on this.      13
Name: sent, dtype: int64




                                    sent flag     simil
577        Pretty sure this is bullshit.    0  0.518113
1630       Wow what a bunch of bullshit.    0  0.516900
1779           This is fucking bullshit.    0  0.514354
592            This is fucking bullshit.    0  0.514354
527           "This is fucking 

In [304]:
for sent in fpm.nlp.pipe(s[s.flag==0].sent.value_counts()[:1000].index.str.lower(), n_process=10, n_threads = 20, disable = ["ner"]):
    matched_keywords = sent[:]._.structurematched
    if matched_keywords:
        print('%s | %s' % (sent, matched_keywords))

bullshit. | ['o']
it’s all bullshit. | ['o']
this is fucking bullshit. | ['svo']
bullshit! | ['o']
fucking bullshit. | ['o']
of course this is total bullshit. | ['svo']
bullshit | ['o']
this is bs. | ['svo']
fucking bullshit | ['o']
bullshit. | ['o']
don't spread bullshit. | ['stvo']
bullshit | ['o']
this is fucking bullshit! | ['svo']
that’s bullshit! | ['o']
*bullshit | ['o']
bullshit... | ['o']
don't spew bullshit. | ['stvo']
this is a load of bs. | ['svo']
bullshit, absolute bullshit. | ['o']
bullshit, bullshit, bullshit | ['o']
this is total bullshit! | ['svo']
bullshit? | ['o']
this is almost assuredly bullshit. | ['svo']
bullshit!! | ['o']
bullshit! | ['o']
it’s all bullshit. | ['o']
fucking bullshit propaganda. | ['o']
it's all bs. | ['svo']
that is bs. | ['svo']
ah, bullshit. | ['o']
*bullshit* | ['o']
everything bullshit. | ['o']
bullshit.. | ['o']
lol just more bullshit | ['o']
that article was nothing but pure bullshit lies. | ['svo']
don’t spread bullshit | ['stvo']
it’s b

In [ ]:
for sent in fpm.nlp.pipe(s[s.flag==0].sent.value_counts()[:50].index.str.lower(), n_process=10, n_threads = 20, disable = ["ner"]):
    matched_keywords = sent[:]._.structurematched
    print('%s | %s' % (sent, matched_keywords))

#### Misleading / Clickbait

In [329]:
s = sent_simil['misleading']
print(s[s.flag==0].sent.value_counts()[:20])
print('\n\n\n')
print(s[s.flag==0][:50])

It could also be from a site that's so steeped in opinion or clickbait, that it can no longer be considered a reliable, fact-based news source.    206
An editorialized, sensationalized, misleading, or factually-inaccurate title will result in a post's removal."                                     185
This is misleading.                                                                                                                                166
It has a sensationalized, editorialized, or biased headline.                                                                                       159
The title of the article is sensationalized or misleading.                                                                                         132
Your post has been removed because the title of the article is sensationalized or misleading.                                                       53
Please read [our headline rules](https://www.reddit.com/r/science/wiki/clickbait) and consider

In [330]:
for sent in fpm.nlp.pipe(s[s.flag==0].sent.value_counts().index[:1000].str.lower(), n_process=10, n_threads = 20, disable = ["ner"]):
    matched_keywords = sent[:]._.structurematched
    if matched_keywords:
        print('%s | %s' % (sent, matched_keywords))

this is misleading. | ['sva']
the title of the article is sensationalized or misleading. | ['o']
very misleading. | ['o']
this is very misleading. | ['sva']
this is a bit misleading. | ['sva']
this is extremely misleading. | ['sva']
editorialized headline | ['ao']
it's misleading. | ['sva']
clickbait. | ['o']
this is so misleading. | ['sva']
clickbait title. | ['ao']
how is it clickbait? | ['o']
this is highly misleading. | ['sva']
no misleading, clickbait, or inaccurate | ['o']
this is really misleading. | ['sva']
it’s not misleading. | ['o']
it is misleading. | ['sva']
this is incredibly misleading. | ['sva']
this is misleading. | ['sva']
this is misleading | ['sva']
super misleading. | ['o']
misleading | ['o']
this is pretty misleading. | ['sva']
this is a little misleading. | ['sva']
completely misleading headline. | ['ao']
a bit misleading. | ['o']
this is slightly misleading. | ['sva']
misleading. | ['o']
this title is misleading and complete and utter bs. | ['sva', 'svo']
that's

In [ ]:
for sent in fpm.nlp.pipe(s[s.flag==0].sent.value_counts().index[:100].str.lower(), n_process=10, n_threads = 20, disable = ["ner"]):
    matched_keywords = sent[:]._.structurematched
    print('%s | %s' % (sent, matched_keywords))

#### Unreliable (source)

In [331]:
s = sent_simil['unreliable']
print(s[s.flag==0].sent.value_counts()[:20])
print('\n\n\n')
print(s[s.flag==0][:50])

Please do not submit links from unreliable, extremely biased, or farcical websites.                                            122
YouTube may not always be a reliable source, especially [unverified or unofficial                                               62
businessinsider.com news source may not unreliable.                                                                             20
Trump is an unreliable source.                                                                                                  16
Bad source (unoriginal, unreliable or satire)                                                                                   14
This post has been removed because either it is from an unreliable source, or the information has already been posted.          12
dailymail.co.uk news source may not unreliable.                                                                                 10
*Posts linking to unreliable sources will be removed.                              

In [334]:
for sent in fpm.nlp.pipe(s[s.flag==0].sent.value_counts().index[:1000].str.lower(), n_process=10, n_threads = 20, disable = ["ner"]):
    matched_keywords = sent[:]._.structurematched
    if matched_keywords:
        print('%s | %s' % (sent, matched_keywords))

that is about as unreliable as any research could be. | ['sva']
it is unreliable. | ['sva']
this is so unreliable. | ['sva']
please don't post unreliable sources like breitbart. | ['stvao']
would that be unreliable? | ['sva']
it's so unreliable | ['sva']
this will be unreliable | ['sva']
this is unreliable. | ['sva']
please don't post unreliable information. | ['stvao']
it's unreliable. | ['sva']
it's unreliable. | ['sva']
&gt;an unreliable source, ... | ['ao']
so it's completely unreliable. | ['sva']
fox news = unreliable source in the service of the white house. | ['ao']
but maybe don't post such an unreliable bias news site. | ['stvao', 'stvao']
yes, it's unreliable - | ['sva']
it's highly unreliable. | ['sva']
this is old and unreliable | ['sva']
it's unreliable | ['sva']
dont spread news from unverified sources. | ['stvao']
this is unreliable, unsubstantiated, and should not be published as fact. | ['sva']
this is wholly unreliable. | ['sva']
if something is unreliable 1/3 of the 

In [ ]:
for sent in fpm.nlp.pipe(s[s.flag==0].sent.value_counts().index[:1000].str.lower(), n_process=10, n_threads = 20, disable = ["ner"]):
    matched_keywords = sent[:]._.structurematched
    print('%s | %s' % (sent, matched_keywords))

#### Propaganda

In [335]:
s = sent_simil['propaganda']
print(s[s.flag==0].sent.value_counts()[:20])
print('\n\n\n')
print(s[s.flag==0][:50])

State-sponsored propaganda is not acceptable on /r/politics.                                                                                      70
[The Rise of the Weaponized AI Propaganda Machine](https://medium.com/join-scout/the-rise-of-the-weaponized-ai-propaganda-machine-86dac61668b)    33
How is this propaganda?                                                                                                                           16
It’s propaganda.                                                                                                                                  15
* is a right wing propaganda outlet disguised as a news aggregator.                                                                               13
It's called propaganda.                                                                                                                           11
How is it propaganda?                                                                                     

In [336]:
for sent in fpm.nlp.pipe(s[s.flag==0].sent.value_counts()[:1000].index, n_process=10, n_threads = 20, disable = ["ner"]):
    matched_keywords = sent[:]._.structurematched
    if matched_keywords:
        print('%s | %s' % (sent, matched_keywords))

Straight up propaganda. | ['o']
Fucking bullshit propaganda. | ['o']
Fucking propaganda | ['o']
Looks like this is another propaganda subreddit masquerading as neutral. | ['svo']
&gt;More bullshit dystopian propaganda. | ['o']
Oh look, more statist propaganda on the sub. | ['vo', 'vao']
Do you like spreading debunked propaganda? | ['stvo']
Do we don't spread our propaganda around the world? | ['stvo', 'stvo']
&gt; Just a right-wing propaganda site. | ['ao']
Please dont spread propaganda. | ['stvo']
It’s not propaganda. | ['o']
Quit spreading bullshit American and western propaganda. | ['stvo']
This is simply fucking propaganda. | ['svo']
&gt;Lmao Western anti-china propaganda | ['o']
*, a right wing propaganda site. | ['ao']
Don’t post this propaganda here. | ['stvo']
Paid propaganda | ['o']
It’s just propaganda at this point | ['o']
It’s not propaganda, but people want to cling to something that will make them feel better. | ['o']
don't post links from propaganda sites please. | ['stv

In [ ]:
for sent in fpm.nlp.pipe(s[s.flag==0].sent.value_counts()[:50].index, n_process=10, n_threads = 20, disable = ["ner"]):
    matched_keywords = sent[:]._.structurematched
    print('%s | %s' % (sent, matched_keywords))

### Non-REGEX

In [13]:
comm_files = sorted(re.findall('COMM_2020_[0-9][0-9].csv', ' '.join(os.listdir(rootfold+'/../output'))))
comm_files

['COMM_2020_01.csv',
 'COMM_2020_02.csv',
 'COMM_2020_03.csv',
 'COMM_2020_04.csv',
 'COMM_2020_05.csv',
 'COMM_2020_06.csv',
 'COMM_2020_07.csv']

In [16]:
comm_file = comm_files[len(comm_files)-1]
comm_generator = pd.read_csv(rootfold+'/../output/'+comm_file, chunksize=100000)
comm_file

'COMM_2020_07.csv'

In [129]:
#for chunk_id, COMM in enumerate(comm_generator):

In [17]:
COMM = next(comm_generator)
COMM = COMM[['subreddit','link_id','id','parent_id','body']]
COMM = COMM[~COMM.id.isin(existing_docids)]
COMM.body = COMM.body.astype(str)

In [246]:
alldocs = list()
for doc_index, doc in enumerate(fpm.nlp.pipe(COMM.body, n_process=10, n_threads = 20, disable = ["tagger", "ner"])):
    sents = [s.text for s in doc.sents]
    for flgtype in flagtypes:
        list_05 = [(doc_index,sent,s,flgtype) \
                   for s,sent in zip([tf_simil(e, mean_flag_embeddings[flgtype]) for e in embed(sents)], sents) \
                   if s>0.5]
        if len(list_05)>0:
            alldocs += list_05
            print(list_05)

[(959, 'bbc.com is a news outlet.', 0.5190278, 'unreliable')]
[(1157, 'the headline is not exact correct', 0.5010978, 'misleading')]
[(1603, '&gt;Post titles must be the exact headline from the article.', 0.51264274, 'misleading')]
[(2010, 'Clickbait nonsense.', 0.52116406, 'misleading')]
[(2307, '(use original, reliable sources).', 0.5171776, 'unreliable'), (2307, "It's not a reliable source.", 0.65313053, 'unreliable')]
[(2447, 'This headline is the opposite of scary..', 0.51329154, 'misleading')]
[(3236, 'The website that you submitted, nypost.com, is not a reliable source.', 0.5575576, 'unreliable')]
[(3655, '(use original, reliable sources).', 0.5171776, 'unreliable'), (3655, "It's not a reliable source.", 0.65313053, 'unreliable')]
[(3711, 'Misleading title again.', 0.519867, 'misleading')]
[(3750, 'Clickbait title', 0.5749717, 'misleading')]
[(4346, 'Misleading title', 0.5373654, 'misleading')]
[(4570, 'Propaganda, basically.', 0.57437134, 'propaganda')]
[(5826, 'Misleading head

[(52659, 'snopes.com is a news outlet.', 0.53307015, 'unreliable')]
[(52734, 'snopes.com is a news outlet.', 0.53307015, 'unreliable')]
[(52820, 'snopes.com is a news outlet.', 0.53307015, 'unreliable')]
[(53442, 'Here is a non Telegraph source:', 0.55731595, 'unreliable')]
[(54793, 'Sees source is The Telegraph', 0.59198475, 'unreliable')]
[(55071, 'Better source:', 0.52784544, 'unreliable')]
[(55954, 'Misleading title.', 0.5373654, 'misleading')]
[(56140, 'I think the post title is a bit click baity.', 0.5122045, 'misleading')]
[(56205, 'Wrong headline.', 0.5054014, 'misleading')]
[(56386, 'You are Fake News!', 0.6598402, 'fakenews')]
[(56386, 'Post from a reputable source.', 0.5491385, 'unreliable')]
[(56394, 'Misleading headline.', 0.5667739, 'misleading')]
[(57592, '(use original, reliable sources).', 0.5171776, 'unreliable'), (57592, "It's not a reliable source.", 0.65313053, 'unreliable')]
[(57653, 'Please be aware that information you receive that does not cite sources may not 

In [251]:
alldocs_df = pd.DataFrame(alldocs, columns = ['id','sent','simil','flgtype'])
alldocs_df.sent.value_counts()

It's not a reliable source.                                                                                                                       29
(use original, reliable sources).                                                                                                                 29
Misleading headline.                                                                                                                               5
The website that you submitted, nypost.com, is not a reliable source.                                                                              3
snopes.com is a news outlet.                                                                                                                       3
CLICKBAIT PIECE OF SHIT                                                                                                                            2
Clickbait title.                                                                                          

In [130]:
comm_simil = {flgtype:pd.DataFrame(columns = ['sent','flag','simil']) for flgtype in flagtypes}
for flgtype in flagtypes:
        print('Started:' + flgtype)
        s = COMM.body
        initial_flags_embedings = embed(s.sent)
        mean_flag_embedding = tf.reduce_mean(embed(s[s.flag>0].sent), axis = 0)
        simildf = pd.DataFrame([(sent,flag,tf_simil(e, mean_flag_embedding)) 
                                     for sent,flag,e in zip(s.sent, s.flag, initial_flags_embedings)
                                     if tf_simil(e, mean_flag_embedding)>0.4], 
                                    columns = ['sent','flag','simil']).\
                        sort_values('simil', ascending=False)
        sent_simil[flgtype] = sent_simil[flgtype].append(simildf)

100000    The idea of people voting worries certain parties
100001    Oh Jesus. We’re going to have President Stern ...
100002    the phones can take, receive calls and go onli...
100003    Policing powers reside with the states, and th...
100004    Like hell no. She is responsible for like 50% ...
                                ...                        
199995    As a black woman, my sleep just got a bit less...
199996    We just borrowed 250 billion, people are argui...
199997    If it hits during the winter I have a feel shi...
199998    This is hardly new. Opposition parties picking...
199999                                      Define "close".
Name: body, Length: 100000, dtype: object

In [252]:
fpm.regex_flag

re.compile(r'\bdisinformation|\bmisinformation|\bmalinformation|\bclickbait|\binnacuracy|\binnacuracies|\bfalsehood|\bfabrication|\bpropaganda|\bbullshit|\bbs|\bnews|\binformation|\binfo|\bfake|\bfalse|\bbogus|\bfabricated|\bmanipulated|\bmanipulative|\binaccurate|\bmisleading|\bmislead|\beditorialized|\bclickbait|\bsensationalized|\buntrustworthy|\bunreliable|\bunverified|\bpropaganda|\bbullshit|\bbs',
re.UNICODE)

In [254]:
COMM[COMM.body.str.contains(fpm.regex_flag)]

,subreddit,link_id,id,parent_id,body
200029,worldnews,t3_gipg7v,t1_fqg0c5y,t3_gipg7v,"This is the best tl;dr I could make, [original..."
200062,metacanada,t3_gioruj,t1_fqg0ane,t3_gioruj,No vaccine for me. The pandemic is not even bi...
200089,politics,t3_gipmpf,t1_fqg09al,t3_gipmpf,Your submission `Neal Took Big Bucks From Lobb...
200091,news,t3_gipmp2,t1_fqg098a,t3_gipmp2,Your post was automatically removed for being ...
200269,Conservative,t3_giop2o,t1_fqg01hy,t3_giop2o,This is just to stop documents from being publ...
...,...,...,...,...,...
299777,australia,t3_ghxqtl,t1_fqc0h0z,t3_ghxqtl,Amazing news! *upvotes* “Legend” *retracts upv...
299837,Coronavirus,t3_gi1dds,t1_fqc0drl,t3_gi1dds,amazon.com is an e-commerce website and isn't ...
299856,CanadaCoronavirus,t3_gi1d5e,t1_fqc0cff,t3_gi1d5e,Thank you for posting to r/CanadaCoronavirus. ...
299906,Coronavirus,t3_ghzxxt,t1_fqc09bx,t3_ghzxxt,I love how this news is in this subreddit ❤️


In [257]:
COMM[COMM.body.str.lower().str.contains(fpm.regex_flag)]

,subreddit,link_id,id,parent_id,body
200029,worldnews,t3_gipg7v,t1_fqg0c5y,t3_gipg7v,"This is the best tl;dr I could make, [original..."
200062,metacanada,t3_gioruj,t1_fqg0ane,t3_gioruj,No vaccine for me. The pandemic is not even bi...
200067,conspiracy,t3_giocci,t1_fqg0afx,t3_giocci,"Explanation rejected. Propaganda is legal, tha..."
200089,politics,t3_gipmpf,t1_fqg09al,t3_gipmpf,Your submission `Neal Took Big Bucks From Lobb...
200091,news,t3_gipmp2,t1_fqg098a,t3_gipmp2,Your post was automatically removed for being ...
...,...,...,...,...,...
299837,Coronavirus,t3_gi1dds,t1_fqc0drl,t3_gi1dds,amazon.com is an e-commerce website and isn't ...
299856,CanadaCoronavirus,t3_gi1d5e,t1_fqc0cff,t3_gi1d5e,Thank you for posting to r/CanadaCoronavirus. ...
299906,Coronavirus,t3_ghzxxt,t1_fqc09bx,t3_ghzxxt,I love how this news is in this subreddit ❤️
299907,Coronavirus,t3_gi10et,t1_fqc09b9,t3_gi10et,"&gt;The oldest woman in Spain, 113 years old, ..."


### Sarcasm and Irony - check

10 jul 2020

In [71]:
comm_files = sorted(re.findall('COMM_2020_[0-9][0-9].csv', ' '.join(os.listdir(rootfold+'/../output'))))
comm_file = comm_files[len(comm_files)-1]
comm_generator = pd.read_csv(rootfold+'/../output/'+comm_file, chunksize=100000)
comm_file

'COMM_2020_07.csv'

In [100]:
C

,id,author,body,collapsed,created,distinguished,edited,index,link_id,no_follow,parent_id,score,subreddit
0,t1_fx1w7eq,Gus-Vdub,Classic bears Lol,False,2020-07-06 00:40:36,NaN,False,1.0,t3_hiefn8,True,t3_hiefn8,1.0,UpliftingNews
1,t1_fx1usjs,ralphtoddsagebenny,"I do wear glasses intentionally, eyes are anot...",False,2020-07-06 00:26:44,NaN,False,8.0,t3_hiq4z5,True,t3_hiq4z5,1.0,Health
2,t1_fx1onka,[deleted],[removed],True,2020-07-05 23:26:17,NaN,False,0.0,t3_hil2p8,True,t3_hil2p8,1.0,Coronavirus
3,t1_fx1m012,jmsatl,I have a theory. That Roberts voted with the l...,False,2020-07-05 23:01:32,NaN,False,0.0,t3_hiop2p,True,t3_hiop2p,1.0,politics
4,t1_fx1j2pd,WarOfNoise,"behold as crime skyrockets, and the police are...",False,2020-07-05 22:34:28,NaN,False,29.0,t3_hiqrja,True,t3_hiqrja,1.0,LosAngeles
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2780319,t1_fsh3bya,AutoModerator,"As a reminder, this subreddit [is for civil di...",False,2020-06-01 00:01:57,moderator,False,2587.0,t3_gu9z9l,True,t3_gu9z9l,1.0,politics
2780320,t1_fsh3b07,AutoModerator,"[](/spikeholdup) Greetings, \/u/HelpPlus! Your...",False,2020-06-01 00:01:42,moderator,False,1248.0,t3_gu9z47,True,t3_gu9z47,1.0,nottheonion
2780321,t1_fsh38je,AutoModerator,"As a reminder, this subreddit [is for civil di...",False,2020-06-01 00:01:02,moderator,False,2596.0,t3_gu9yqz,True,t3_gu9yqz,1.0,politics
2780322,t1_fsh36hs,AutoModerator,"[](/spikeholdup) Greetings, \/u/HelpPlus! Your...",False,2020-06-01 00:00:30,moderator,False,1249.0,t3_gu9yeu,True,t3_gu9yeu,1.0,nottheonion


In [92]:
sarcasm_regex = "\\\s\\b|\\/s\\b"  
irony_regex = '|'.join(["[*'\"]" + s for s in fpm.voc.attr.false + fpm.voc.attr.propaganda + fpm.voc.obj.disinfo])
joking_regex = '\\bjk\\b'
sarcasm_and_irony_regex = sarcasm_regex + '|' + irony_regex + '|' + joking_regex
sarcasm_and_irony_regex 

'\\\\s\\b|\\/s\\b|[*\'"]fake|[*\'"]false|[*\'"]bogus|[*\'"]fabricated|[*\'"]manipulated|[*\'"]manipulative|[*\'"]inaccurate|[*\'"]propaganda|[*\'"]disinformation|[*\'"]misinformation|[*\'"]malinformation|\\bjk\\b'

In [93]:
artificial = pd.DataFrame({'body':['asd xxx','xxxdssa','Thanks Obama \s','Trump smart /s', 'Not related asd/s da',
                                   'I am sick of fake news',"I am sick of 'fake' news",'This "disinformation" must stop',
                                  '*fake* news is fake','idk jk','sadasdjkdsa']})
artificial['contains'] = artificial.body.str.contains(sarcasm_and_irony_regex)
artificial

,body,contains
0,asd xxx,False
1,xxxdssa,False
2,Thanks Obama \s,True
3,Trump smart /s,True
4,Not related asd/s da,True
5,I am sick of fake news,False
6,I am sick of 'fake' news,True
7,"This ""disinformation"" must stop",True
8,*fake* news is fake,True
9,idk jk,True


In [83]:
COMM = next(comm_generator)
COMM = COMM[['subreddit','link_id','id','parent_id','body']]
COMM.body = COMM.body.astype(str)

In [86]:
test = COMM.body[:1000000]
n=0
for txt in test[test.str.lower().str.contains(sarcasm_and_irony_regex)]:
    if len(txt)<200:
        print(txt)
        print('\n\n')
    n+=1
print(n)

Oh yeah, and then they'll turn up the 5g towers and fry everyone /s



Maybe that’s why he said it will just disappear, coz it was made in China /s



Where I live all the mangroves have been replaced with mansions. I'm sure their seawalls will work just as well. /s



He also makes free toilets for Africa. I think the toilets inject a mind control chip in your ass while you shit. /s



The Chinese sent us a fake virus. But then people started believing in it, so it became real. It’s the Tinker Bell effect, duh! /s



Anti-semite! /s



Just quit testing /s



Sad that he has to live in US now /s



Yup, definitely was social media. If the US didn't have social media like the rest of the world, this wouldn't have happened. /S Look deeper Bill.



Oh, I get it. So it's China's fault we won't wear masks and opened up the bars. Gotcha. /s



The difference is monkeys cant pull theirselves up by their bootstraps! /s



China was also responsible for the muted response from the Trump admini

In [70]:
COMM

,subreddit,link_id,id,parent_id,body
0,science,t3_ho1q3u,t1_fxhihnb,t3_ho1q3u,Different story all the time. Maybe this is a ...
1,science,t3_ho6ojw,t1_fxhi3t0,t3_ho6ojw,Sounds extremely interesting as a concept. Gat...
2,trump,t3_hofqmw,t1_fxhi27o,t3_hofqmw,"FTO: ""There is, of course, absolutely no way t..."
3,trump,t3_hofqmw,t1_fxhhu82,t3_hofqmw,This subreddit is a pro-Trump subreddit for sh...
4,ukpolitics,t3_hncehd,t1_fxhht52,t3_hncehd,have those in no10 had brain scans lately?
...,...,...,...,...,...
99995,singapore,t3_hmthyh,t1_fx7r37j,t3_hmthyh,Another Great Crony.
99996,apple,t3_hmubh1,t1_fx7r36d,t3_hmubh1,[Here is a picture](https://postimg.cc/kVqJZpG...
99997,singapore,t3_hmuagr,t1_fx7r2zm,t1_fx7bq3j,"They’re paid help, not volunteer workers. It’s..."
99998,europe,t3_hm8kgq,t1_fx7r2yt,t3_hm8kgq,"Hard to feel sorry for them, but a House in yo..."


In [103]:
COMM.body.str.contains(fpm.bot_author_regex)

AttributeError: module 'functions_pos_match' has no attribute 'bot_author_regex'

In [20]:
COMM.body.str.contains('story')

0         True
1        False
2        False
3        False
4        False
         ...  
99995    False
99996    False
99997    False
99998    False
99999    False
Name: body, Length: 100000, dtype: bool